In [9]:
import rasterio
import glob, cv2
import numpy as np

In [10]:
fp_dir = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img"
list_fp = glob.glob(os.path.join(fp_dir, '*.tif'))

In [11]:
fp_img = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\48U_20210101-20220101_0.tif"
fp_flow = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\LC08_L2SP_133026_20210201_20210302_02_T1_0.tif"
fp_out = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\resize1.tif"

In [12]:
img_flow = rasterio.open(fp_flow)
width = rasterio.open(fp_flow).width
height = rasterio.open(fp_flow).height

img1 = cv2.imread(fp_img, cv2.IMREAD_UNCHANGED)
new_shape = (width, height)
resized = cv2.resize(img1, new_shape, interpolation = cv2.IMREAD_UNCHANGED)
resized.shape

RasterioIOError: E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\LC08_L2SP_133026_20210201_20210302_02_T1_0.tif: No such file or directory

In [ ]:
profile = rasterio.open(fp_flow).profile
profile.update( count=1,
                dtype=rasterio.uint8
                )
with rasterio.open(fp_out, 'w', **profile) as dst:
            dst.write(np.array([resized]))

In [ ]:
import os
import glob
import rasterio
import numpy as np

from osgeo import gdal
from rasterio.warp import reproject, Resampling


def window_from_extent(xmin, xmax, ymin, ymax, aff):
        col_start, row_start = ~aff * (xmin, ymax)
        col_stop, row_stop = ~aff * (xmax, ymin)
        return ((int(row_start), int(row_stop)), (int(col_start), int(col_stop)))

def convert_profile(src_path, dst_path, out_path):
    kwargs = None
    _info = gdal.Info(dst_path, format='json')
    xmin, ymin = _info['cornerCoordinates']['lowerLeft']
    xmax, ymax = _info['cornerCoordinates']['upperRight']

    with rasterio.open(dst_path) as dst:
        dst_transform = dst.transform
        kwargs = dst.meta
        kwargs['transform'] = dst_transform
        dst_crs = dst.crs
        kwargs.update(count=1,
                      dtype=rasterio.uint8)

    with rasterio.open(src_path) as src:
        window = window_from_extent(xmin, xmax, ymin, ymax, src.transform)
        src_transform = src.window_transform(window)
        data = src.read()
        print(np.unique(data))

        with rasterio.open(out_path, 'w', **kwargs) as dst:
            # for i, band in enumerate(data, 1):
            # data = src.read(window=window)
            print(np.unique(data))
            # print(data.shape)
            dest = np.zeros_like(data) 
            reproject(
                data, dest,
                src_transform=src_transform,
                src_crs=src.crs,
                dst_transform=src_transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
            print(np.unique(dest))
            dst.write(dest)

In [ ]:
src_path = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\resize_near.tif"
dst_path = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\LC08_L2SP_133026_20210201_20210302_02_T1_0.tif"
out_path = r"E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\resize_near_ext.tif"
convert_profile(src_path, dst_path, out_path)

[ 0  1  2  4  5  7  8 11]
[ 0  1  2  4  5  7  8 11]
[ 0  1  2  4  5  7  8 11]


In [ ]:
!gdalwarp -of GTiff -co COMPRESS=DEFLATE -ts 1391 862 -r near "E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\48U_20210101-20220101_0.tif" "E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\resize_near.tif"

Creating output file that is 1391P x 862L.
Processing E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\48U_20210101-20220101_0.tif [1/1] : 0Using internal nodata values (e.g. 0) for image E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\48U_20210101-20220101_0.tif.
Copying nodata values from source E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\48U_20210101-20220101_0.tif to destination E:\WORK\Mongodia\Data_cut\tmp\Data_cut_img\resize_near.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:

# Append Pandas DataFrame to Existing CSV File
# importing pandas module
import pandas as pd
 
# data of Player and their performance
data = {
    'Name': ['Hardik', 'Pollard', 'Bravo'],
    'Run': [50, 63, 15],
    'Wicket': [0, 2, 3],
    'Catch': [4, 2, 1]
}
 
# Make data frame of above data
df = pd.DataFrame(data)
 
# append data frame to CSV file
df.to_csv('GFG.csv', mode='a', index=False, header=False)
 
# print message
print("Data appended successfully.")

Data appended successfully.


In [13]:
import rasterio
import rasterio.features
import os
import geopandas as gp
import numpy as np
from matplotlib import pyplot as plt
import cv2
import argparse
import sys
import glob, os
from multiprocessing.pool import Pool
from functools import partial
import multiprocessing
import time
core = multiprocessing.cpu_count()


In [14]:
def create_mask_by_shapefile2(shape_path, height, width, tr):
    # shape_file = os.path.expanduser(shape_path)
    # list_shape = create_list_id(shape_path,'.shp')
    list_geometry = []
    # for shape_id in list_shape: 
    #     shape_file = os.path.join(shape_path,shape_id+'.shp')
    shp = gp.read_file(shape_path)
    ls_geo = [(x.geometry) for i, x in shp.iterrows()]
    list_geometry.extend(ls_geo)
    mask = rasterio.features.rasterize(list_geometry
                                    ,out_shape=(height, width)
                                    ,transform=tr)
    return mask

In [15]:
def arr2raster(path_out, bands, height, width, tr, dtype="uint8",coordinate=None,projstr=None):
    num_band = len(bands)
    if coordinate!= None:
        crs = rasterio.crs.CRS.from_epsg(coordinate)
    else:
        crs = rasterio.crs.CRS.from_string(projstr)
    new_dataset = rasterio.open(path_out, 'w', driver='GTiff',
                            height = height, width = width,
                            count = num_band, dtype = dtype,
                            crs = crs,
                            transform = tr,
                            # nodata = 0,
                            compress='lzw')
    if num_band == 1:
        new_dataset.write(bands[0], 1)
    else:
        for i in range(num_band):
            new_dataset.write(bands[i],i+1)
    new_dataset.close()

In [31]:
def build_mask(path_image, dir_fp_shp, output_mask):
    path_image = path_image
    list_fp_shp =  glob.glob(os.path.join(dir_fp_shp,'*.shp'))
    list_fp_shp.sort()
    print(list_fp_shp)
    with rasterio.open(path_image) as src:
        tr = src.transform
        w,h = src.width,src.height
        projstr = (src.crs.to_string())
        print(projstr)
        crs = src.crs
        check_epsg = crs.is_epsg_code
        # if check_epsg:
        coordinate = src.crs.to_epsg()
        # else:
        # coordinate = None
        mask = np.zeros_like(src.read())[0]
    i=1
    for path_shp in list_fp_shp:
        mask += create_mask_by_shapefile2(path_shp, h, w, tr)*i
        i+=1
    mask = mask.astype('uint8')
    arr2raster(output_mask, [mask], h, w, tr, dtype="uint8",coordinate=coordinate,projstr=projstr)

In [32]:
path_image= r"E:\WORK\Mongodia\Data_cut_img\Img\LC08_L2SP_133026_20210201_20210302_02_T1_0.tif"
dir_fp_shp = r"E:\WORK\Mongodia\Data_cut_img\label_shp"
output_mask = r"E:\WORK\Mongodia\Data_cut_img\label_shp\mask.tif"
build_mask(path_image, dir_fp_shp, output_mask)

['E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\1_build_up.shp', 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\2_crop.shp', 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\3_other.shp', 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\4_pasture.shp', 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\5_tree.shp', 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\6_water.shp']
EPSG:32648


In [24]:
dir_fp_shp = r"E:\WORK\Mongodia\Data_cut_img\label_shp"
a = glob.glob(os.path.join(dir_fp_shp,'*.shp'))

In [25]:
a

['E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\1_build_up.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\2_crop.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\3_other.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\4_pasture.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\5_tree.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\6_water.shp']

In [27]:
a.sort()

In [28]:
a

['E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\1_build_up.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\2_crop.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\3_other.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\4_pasture.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\5_tree.shp',
 'E:\\WORK\\Mongodia\\Data_cut_img\\label_shp\\6_water.shp']

In [16]:
import numpy as np
a=np.random.rand(4,3,2)

In [20]:
a[:, 0:,:]

array([[[0.6827058 , 0.48579679],
        [0.38128813, 0.61612236],
        [0.04059653, 0.42403218]],

       [[0.96674929, 0.12238189],
        [0.31954878, 0.8677101 ],
        [0.13945541, 0.52228638]],

       [[0.88074106, 0.696703  ],
        [0.03074569, 0.85945163],
        [0.15704265, 0.09845663]],

       [[0.94720132, 0.65945073],
        [0.48966806, 0.13855657],
        [0.66585167, 0.44710311]]])

In [18]:
a

array([[[0.6827058 , 0.48579679],
        [0.38128813, 0.61612236],
        [0.04059653, 0.42403218]],

       [[0.96674929, 0.12238189],
        [0.31954878, 0.8677101 ],
        [0.13945541, 0.52228638]],

       [[0.88074106, 0.696703  ],
        [0.03074569, 0.85945163],
        [0.15704265, 0.09845663]],

       [[0.94720132, 0.65945073],
        [0.48966806, 0.13855657],
        [0.66585167, 0.44710311]]])

In [4]:
import rasterio
import glob, os

dir_path = r"C:\Users\SkyMap\Desktop\NDA\mosaic"
list_img_name = glob.glob(os.path.join(dir_path,"*.tif"))
with rasterio.open(list_img_name[0]) as src:
    crs_flow = src.crs
for fp in list_img_name[1:]:
    src = rasterio.open(fp).crs
    if crs_flow == src:
        print("a")
    # print(src)

